In [2]:
!pip install -q langchain langchain-core langchain-huggingface python-dotenv

In [3]:
!pip install -q huggingface_hub


In [6]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""


In [8]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import os

# ✅ Set token manually if .env not working
# (skip if already set)
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_xxxxx"

# Load env (optional if .env present)
load_dotenv()

# Initialize the Hugging Face model endpoint
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")  # ✅ explicitly passed
)

# Wrap it in a Chat interface
model = ChatHuggingFace(llm=llm)

# Define schema for structured output
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

# Create the prompt
template = PromptTemplate(
    template='Give 3 facts about {topic} \n{format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

# Build and execute the chain
chain = template | model | parser

result = chain.invoke({'topic': 'ai engineer 50lpa'})

print(result)

{'fact_1': 'AI engineers with a specialization in Natural Language Processing (NLP) can command salaries exceeding ₹50 lakh per annum.', 'fact_2': 'The demand for skilled AI engineers is rapidly growing, significantly contributing to higher compensation packages in the sector.', 'fact_3': 'Compensation for AI engineers may vary based on factors such as experience, location, and company size, with specialization, skills and proven track record playing a decisive role.'}
